# Context data for the sea level data explorer

In [1]:
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime

#get data from worldbank
from pandas_datareader import wb

import glob
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import json
import matplotlib.pyplot as plt

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 200
#matplotlib.rcParams['svg.fonttype'] = 'none'


# Sea Level Data, Stations

In [19]:
stations = pd.read_csv('context_data/sealevel_viz_psmsl_1985_2015.csv', usecols=[2, 4, 5, 6, 7, 9])

stations = stations.drop_duplicates()

stations.rename(columns={'ID': 'id'}, inplace=True)

stations['id'].nunique()
stations.head()

513

,id,location,latitude,longitude,country,continent
0,1,Brest,48.383,-4.495,FRA,Europe
31,61,Marseille,43.279,5.354,FRA,Europe
62,453,Le Havre,49.482,0.106,FRA,Europe
93,467,Cherbourg,49.651,-1.636,FRA,Europe
124,468,Dunkerque,51.048,2.367,FRA,Europe


# Worldbank Data, Emissions

In [24]:
#get the data from worldbank, Stand: 7. April 2017

emissions = pd.read_csv('context_data/c02_emissions_worlbank.csv', usecols=[0, 1, 57])

emissions['c02_rank'] = emissions['2013'].rank(ascending=False)
emissions.rename(columns={'Country Code': 'country_code'}, inplace=True)
emissions.rename(columns={'Country Name': 'country_name'}, inplace=True)
emissions.rename(columns={'2013': 'c02_emissions'}, inplace=True)

emissions.sort_values(by='c02_rank').head()

,country_name,country_code,c02_emissions,c02_rank
198,Qatar,QAT,40.462355,1.0
240,Trinidad and Tobago,TTO,34.520237,2.0
49,Curacao,CUW,34.161635,3.0
125,Kuwait,KWT,27.258964,4.0
20,Bahrain,BHR,23.682574,5.0


# Sea Level Population Data, National Population Data

In [9]:
sealevel_pop = pd.read_csv('context_data/lecz-urban-rural-population-land-area-estimates_country-90m.csv')

In [10]:
sealevel_pop['ElevationZone'] = sealevel_pop['ElevationZone'].str.strip()
sealevel_pop['ISO3v10'] = sealevel_pop['ISO3v10'].str.strip()
pop = sealevel_pop.set_index(['ElevationZone'])
pop = pop.loc[['Elevations Less Than or Equal To 10m']]

pop = pop.reset_index()

pop_low_elevation = pop.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_low_elevation = pop_low_elevation.to_frame(name = 'coastal_population2010_sum')
pop_low_elevation = pop_low_elevation.reset_index()

pop_low_elevation.head()

,ISO3v10,coastal_population2010_sum
0,ABW,21327
1,AGO,651360
2,AIA,3848
3,ALB,354132
4,ANT,46295


In [11]:
pop_total = sealevel_pop.set_index(['ElevationZone'])
pop_total = pop_total.loc[['Total National Population']]

pop_total = pop_total.reset_index()

pop_total = pop_total.groupby(by=['ISO3v10'])['Population2010'].sum()
pop_total = pop_total.to_frame(name = 'total_population2010_sum')
pop_total = pop_total.reset_index()

pop_total.head()

,ISO3v10,total_population2010_sum
0,ABW,108141
1,AGO,19618432
2,AIA,15645
3,ALB,3215988
4,ANT,202847


# Sea Level PSMSL Trend Data

In [13]:
trend_1985_2015 = pd.read_csv('context_data/trends_1985_2015.csv', usecols=[4,7])
trend_1985_2015.rename(columns={'trend': 'trend_1985_2015'}, inplace=True)
trend_1985_2015.head()

,trend_1985_2015,_id
0,2.385,1
1,3.531,7
2,1.476,8
3,1.484,9
4,2.190,10


In [21]:
trend_longest = pd.read_csv('context_data/trends_longest.csv', usecols=[4, 6, 12])
trend_longest.rename(columns={'trend': 'trend_longest'}, inplace=True)
trend_longest.rename(columns={'startYear': 'start_year'}, inplace=True)
trend_longest.head()

,start_year,trend_longest,_id
0,1807,0.97,1
1,1811,0.80,2
2,1968,1.73,3
3,1843,2.12,7
4,1849,1.42,8


# Risk Level Data

In [29]:
riskindex = pd.read_csv('context_data/riskindex.csv', usecols=[0,1,2,3])
riskindex.rename(columns={'Rank': 'risk_rank'}, inplace=True)
riskindex.head()

,risk_rank,Country,risk_index,risk_description
0,1,Vanuatu,36.28,very high
1,2,Tonga,29.33,very high
2,3,Philippines,26.70,very high
3,4,Guatemala,19.88,very high
4,5,Bangladesh,19.17,very high


# Create Sea Level Context Data

In [30]:
#Add emissions
sealevel_context = stations.merge(emissions, left_on='country', right_on='country_code')

#Add sealevel population
sealevel_context = sealevel_context.merge(pop_low_elevation, left_on='country', right_on='ISO3v10')

#Add national population
sealevel_context = sealevel_context.merge(pop_total, left_on='ISO3v10', right_on='ISO3v10')

#Add trend data for 1985 to 2015
sealevel_context = sealevel_context.merge(trend_1985_2015, left_on='id', right_on='_id')

#Add longest trend
sealevel_context = sealevel_context.merge(trend_longest, left_on='_id', right_on='_id')

#Add riskindex
sealevel_context = sealevel_context.merge(riskindex, left_on='country_name', right_on='Country')

sealevel_context.head()

,id,location,latitude,longitude,country,continent,country_name,country_code,c02_emissions,c02_rank,...,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,_id,start_year,trend_longest,risk_rank,Country,risk_index,risk_description
0,1,Brest,48.383,-4.495,FRA,Europe,France,FRA,5.050483,92.0,...,2983590,63125894,2.385,1,1807,0.97,152,France,2.62,very low
1,61,Marseille,43.279,5.354,FRA,Europe,France,FRA,5.050483,92.0,...,2983590,63125894,3.530,61,1885,1.32,152,France,2.62,very low
2,453,Le Havre,49.482,0.106,FRA,Europe,France,FRA,5.050483,92.0,...,2983590,63125894,1.984,453,1959,2.66,152,France,2.62,very low
3,467,Cherbourg,49.651,-1.636,FRA,Europe,France,FRA,5.050483,92.0,...,2983590,63125894,1.908,467,1975,1.47,152,France,2.62,very low
4,1247,Port Tudy,47.644,-3.446,FRA,Europe,France,FRA,5.050483,92.0,...,2983590,63125894,2.971,1247,1976,2.47,152,France,2.62,very low


In [31]:
sealevel_context = sealevel_context.drop('Country', 1)
sealevel_context = sealevel_context.drop('country', 1)
sealevel_context = sealevel_context.drop('ISO3v10', 1)
sealevel_context = sealevel_context.drop('_id', 1)

In [32]:
sealevel_context.head()

,id,location,latitude,longitude,continent,country_name,country_code,c02_emissions,c02_rank,coastal_population2010_sum,total_population2010_sum,trend_1985_2015,start_year,trend_longest,risk_rank,risk_index,risk_description
0,1,Brest,48.383,-4.495,Europe,France,FRA,5.050483,92.0,2983590,63125894,2.385,1807,0.97,152,2.62,very low
1,61,Marseille,43.279,5.354,Europe,France,FRA,5.050483,92.0,2983590,63125894,3.530,1885,1.32,152,2.62,very low
2,453,Le Havre,49.482,0.106,Europe,France,FRA,5.050483,92.0,2983590,63125894,1.984,1959,2.66,152,2.62,very low
3,467,Cherbourg,49.651,-1.636,Europe,France,FRA,5.050483,92.0,2983590,63125894,1.908,1975,1.47,152,2.62,very low
4,1247,Port Tudy,47.644,-3.446,Europe,France,FRA,5.050483,92.0,2983590,63125894,2.971,1976,2.47,152,2.62,very low


In [33]:
sealevel_context.to_csv('context_data/sealevel_context_data.csv')